### This Notebook takes the previously created BC ignoring the NEMO Levels and JP practical salinity and does the vertical interpolation and gsw calls to generate proper salinity to write them into final input BC file one for each boundary

In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import matplotlib.cm as cm
from scipy.interpolate import interp1d
from salishsea_tools import (nc_tools, gsw_calls,viz_tools)

##### Import the mesh_mask.nc for the depth levels on which to interpolate our values (We will use the mask recently created with 27 depth levels after playing with the NEMO parameters)

In [16]:
mask = nc.Dataset('/ocean/ssahu/CANYONS/Runs/trial_run_mesh_mask_gen/mesh_mask.nc')

#mask = nc.Dataset('/ocean/ssahu/CANYONS/Results/Westcoast_attempt17_generate_mask_80_levels/mesh_mask.nc')

NEMO_depth_T = mask.variables['nav_lev'][:];
NEMO_depth_U = mask.variables['gdepw_0'][0,:,0,0];

print(NEMO_depth_T.shape, NEMO_depth_U.shape)

(27,) (27,)


In [18]:
NEMO_depth_T, NEMO_depth_U

(array([   14.75075722,    24.42877769,    34.46412659,    45.19285583,
           57.26233673,    71.90678406,    91.43279266,   119.99002075,
          164.52903748,   235.26014709,   344.00537109,   499.3053894 ,
          701.16015625,   941.02905273,  1207.08996582,  1489.1328125 ,
         1780.20678711,  2076.16235352,  2374.69287109,  2674.56420898,
         2975.12866211,  3276.05078125,  3577.15649414,  3878.35668945,
         4179.60546875,  4480.87939453,  4782.16601562], dtype=float32),
 array([    0.        ,    19.55859375,    29.38588524,    39.71109009,
           51.00104904,    64.15120697,    80.85449219,   104.22452545,
          139.69599915,   195.87712097,   284.19564819,   415.6000061 ,
          594.7956543 ,   817.07714844,  1071.49597168,  1346.62451172,
         1633.85449219,  1927.7512207 ,  2225.20117188,  2524.51098633,
         2824.78588867,  3125.55859375,  3426.58764648,  3727.74853516,
         4028.97705078,  4330.24023438,  4631.52148438], dtype=

#### Let us start with the left boundary (West)

##### The 2D output BC fields are fine so we donot need to worry about them and they have been properly created by the previous notebook (Writing_bdy_files) but the fields inside the 3d boundary files have not been interpolated and the salinity values need to be converted to absolute salinity from PSU

In [19]:
left_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_west_m04.nc');

vosaline_JP_level_PSU_left = left_file.variables['vosaline'][:];
votemper_JP_level_PT_left = left_file.variables['votemper'][:];
U_vel_JP_level_left = left_file.variables['vozocrtx'][:];
V_vel_JP_level_left = left_file.variables['vomecrty'][:];


print(vosaline_JP_level_PSU_left.shape, votemper_JP_level_PT_left.shape, U_vel_JP_level_left.shape, V_vel_JP_level_left.shape)

(43, 50, 4, 100) (43, 50, 4, 100) (43, 50, 4, 100) (43, 50, 4, 100)


#### Our boundary files donot have a depth variable from which we can import the depths but the files created before the vertical interpolation of the IC values have the depth variable (the file is called West_coast_temperature_salinity_nomask_JP.nc)

In [20]:
depth_JP_T = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/mesh_mask.nc').variables['gdept_0'][0,:,0,0];
depth_JP_U = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/mesh_mask.nc').variables['gdepw_0'][0,:,0,0];
lat = nc.Dataset('/ocean/ssahu/CANYONS/wcvi/initial_conditions/West_coast_temperature_salinity_nomask_JP.nc')\
            .variables['nav_lat'][:];
lon = nc.Dataset('/ocean/ssahu/CANYONS/wcvi/initial_conditions/West_coast_temperature_salinity_nomask_JP.nc')\
            .variables['nav_lon'][:]
print(depth_JP_T.shape, depth_JP_U.shape, lat.shape)

(50,) (50,) (100, 70)


In [21]:
# Conversion of vosaline_PSU to vosaline_RS and votemper_PT to votemper_CT

z = np.dot(-1,depth_JP_T);

pressure = np.zeros(z.shape);
lats = np.zeros(pressure.shape);
lons = np.zeros(pressure.shape);
lats[:] = np.mean(lat);
lons[:] = np.mean(lon);

vosaline_JP_level_SA_left = np.zeros(vosaline_JP_level_PSU_left.shape);
vosaline_JP_level_RS_left = np.zeros(vosaline_JP_level_PSU_left.shape);
votemper_JP_level_CT_left = np.zeros(vosaline_JP_level_PSU_left.shape);

pressure = gsw_calls.generic_gsw_caller('gsw_p_from_z.m', [z, np.mean(lat)]);
for i,j in enumerate(vosaline_JP_level_SA_left[0,:,...]):
    vosaline_JP_level_SA_left[:,i,...] = gsw_calls.generic_gsw_caller('gsw_SA_from_SP', [vosaline_JP_level_PSU_left[:,i,...],pressure[i],lons[i],lats[i]]);

vosaline_JP_level_RS_left[:] = gsw_calls.generic_gsw_caller('gsw_SR_from_SP', [vosaline_JP_level_PSU_left[:]]);
votemper_JP_level_CT_left[:] = gsw_calls.generic_gsw_caller('gsw_CT_from_pt', [vosaline_JP_level_SA_left[:], votemper_JP_level_PT_left[:]]);

In [22]:
vosaline_JP_level_RS_left.shape

(43, 50, 4, 100)

In [24]:
#### let us interp1d to make a vertical function for each of these variables and then interpolate them to the values vertically

left_temp_function = interp1d(depth_JP_T, votemper_JP_level_CT_left, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
left_sal_function = interp1d(depth_JP_T, vosaline_JP_level_RS_left, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
left_U_function = interp1d(depth_JP_U, U_vel_JP_level_left, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
left_V_function = interp1d(depth_JP_U, V_vel_JP_level_left, axis=1,\
                              bounds_error=False, fill_value='extrapolate');

In [27]:
votemper_NEMO_left = np.zeros((vosaline_JP_level_SA_left.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_left.shape[2], vosaline_JP_level_SA_left.shape[3]));
vosaline_NEMO_left = np.zeros((vosaline_JP_level_SA_left.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_left.shape[2], vosaline_JP_level_SA_left.shape[3]));
U_NEMO_left = np.zeros((vosaline_JP_level_SA_left.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_left.shape[2], vosaline_JP_level_SA_left.shape[3]));
V_NEMO_left = np.zeros((vosaline_JP_level_SA_left.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_left.shape[2], vosaline_JP_level_SA_left.shape[3]));


for indx in np.arange(NEMO_depth_T.shape[0]):
    votemper_NEMO_left[:,indx,...] = left_temp_function(NEMO_depth_T[indx]);
    vosaline_NEMO_left[:,indx,...] = left_sal_function(NEMO_depth_T[indx]);
    U_NEMO_left[:,indx,...]        = left_U_function(NEMO_depth_U[indx]);
    V_NEMO_left[:,indx,...]        = left_V_function(NEMO_depth_U[indx]);

In [28]:
print(votemper_NEMO_left.shape, vosaline_NEMO_left.shape, U_NEMO_left.shape, V_NEMO_left.shape)

(43, 27, 4, 100) (43, 27, 4, 100) (43, 27, 4, 100) (43, 27, 4, 100)


In [31]:
#Now let us write the 3d boundary condition for the left boundary

bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_west_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xb', U_NEMO_left.shape[3]);
bdy_file.createDimension('yb', U_NEMO_left.shape[2]);
bdy_file.createDimension('deptht', U_NEMO_left.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along left boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of left boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = U_NEMO_left[:];
vomecrty[:] = V_NEMO_left[:];
votemper[:] = votemper_NEMO_left[:];
vosaline[:] = vosaline_NEMO_left[:];
deptht[:] = NEMO_depth_T[:];

bdy_file.close()

### Now for the right boundary

In [34]:
# for the right boundary I will make everything occur within one cell so that eventually it will be easier to automate it

right_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_right_m04.nc');

vosaline_JP_level_PSU_right = right_file.variables['vosaline'][:];
votemper_JP_level_PT_right = right_file.variables['votemper'][:];
U_vel_JP_level_right = right_file.variables['vozocrtx'][:];
V_vel_JP_level_right = right_file.variables['vomecrty'][:];


print(vosaline_JP_level_PSU_right.shape, votemper_JP_level_PT_right.shape, U_vel_JP_level_right.shape, V_vel_JP_level_right.shape)

vosaline_JP_level_SA_right = np.zeros(vosaline_JP_level_PSU_right.shape);
vosaline_JP_level_RS_right = np.zeros(vosaline_JP_level_PSU_right.shape);
votemper_JP_level_CT_right = np.zeros(vosaline_JP_level_PSU_right.shape);


pressure = gsw_calls.generic_gsw_caller('gsw_p_from_z.m', [z, np.mean(lat)]);

for i,j in enumerate(vosaline_JP_level_SA_right[0,:,...]):
    vosaline_JP_level_SA_right[:,i,...] = gsw_calls.generic_gsw_caller(\
                                    'gsw_SA_from_SP', [vosaline_JP_level_PSU_right[:,i,...],pressure[i],lons[i],lats[i]]);

vosaline_JP_level_RS_right[:] = gsw_calls.generic_gsw_caller('gsw_SR_from_SP', [vosaline_JP_level_PSU_right[:]]);
votemper_JP_level_CT_right[:] = gsw_calls.generic_gsw_caller('gsw_CT_from_pt', [vosaline_JP_level_SA_right[:], votemper_JP_level_PT_right[:]]);    
    
print(vosaline_JP_level_SA_right.shape)


right_temp_function = interp1d(depth_JP_T, votemper_JP_level_CT_right, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
right_sal_function = interp1d(depth_JP_T, vosaline_JP_level_RS_right, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
right_U_function = interp1d(depth_JP_U, U_vel_JP_level_right,axis=1,\
                              bounds_error=False, fill_value='extrapolate');
right_V_function = interp1d(depth_JP_U, V_vel_JP_level_right, axis=1,\
                              bounds_error=False, fill_value='extrapolate');


votemper_NEMO_right = np.zeros((vosaline_JP_level_SA_right.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_right.shape[2], vosaline_JP_level_SA_right.shape[3]));
vosaline_NEMO_right = np.zeros((vosaline_JP_level_SA_right.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_right.shape[2], vosaline_JP_level_SA_right.shape[3]));
U_NEMO_right = np.zeros((vosaline_JP_level_SA_right.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_right.shape[2], vosaline_JP_level_SA_right.shape[3]));
V_NEMO_right = np.zeros((vosaline_JP_level_SA_right.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_right.shape[2], vosaline_JP_level_SA_right.shape[3]));


for indx in np.arange(NEMO_depth_T.shape[0]):
    votemper_NEMO_right[:,indx,...] = right_temp_function(NEMO_depth_T[indx]);
    vosaline_NEMO_right[:,indx,...] = right_sal_function(NEMO_depth_T[indx]);
    U_NEMO_right[:,indx,...]        = right_U_function(NEMO_depth_U[indx]);
    V_NEMO_right[:,indx,...]        = right_V_function(NEMO_depth_U[indx]);

    
print(votemper_NEMO_right.shape, vosaline_NEMO_right.shape, U_NEMO_right.shape, V_NEMO_right.shape)


#Now let us write the 3d boundary condition for the right boundary

bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_right_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xb', U_NEMO_right.shape[3]);
bdy_file.createDimension('yb', U_NEMO_right.shape[2]);
bdy_file.createDimension('deptht', U_NEMO_right.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along right boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of right boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = U_NEMO_right[:];
vomecrty[:] = V_NEMO_right[:];
votemper[:] = votemper_NEMO_right[:];
vosaline[:] = vosaline_NEMO_right[:];
deptht[:] = NEMO_depth_T[:];

bdy_file.close()

(43, 50, 4, 100) (43, 50, 4, 100) (43, 50, 4, 100) (43, 50, 4, 100)
(43, 50, 4, 100)
(43, 27, 4, 100) (43, 27, 4, 100) (43, 27, 4, 100) (43, 27, 4, 100)


### Now for the north boundary

In [35]:
north_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_north_m04.nc');

vosaline_JP_level_PSU_north = north_file.variables['vosaline'][:];
votemper_JP_level_PT_north = north_file.variables['votemper'][:];
U_vel_JP_level_north = north_file.variables['vozocrtx'][:];
V_vel_JP_level_north = north_file.variables['vomecrty'][:];


print(vosaline_JP_level_PSU_north.shape, votemper_JP_level_PT_north.shape, U_vel_JP_level_north.shape, V_vel_JP_level_north.shape)

vosaline_JP_level_SA_north = np.zeros(vosaline_JP_level_PSU_north.shape);
vosaline_JP_level_RS_north = np.zeros(vosaline_JP_level_PSU_north.shape);
votemper_JP_level_CT_north = np.zeros(vosaline_JP_level_PSU_north.shape);

pressure = gsw_calls.generic_gsw_caller('gsw_p_from_z.m', [z, np.mean(lat)]);

for i,j in enumerate(vosaline_JP_level_SA_north[0,:,...]):
    vosaline_JP_level_SA_north[:,i,...] = gsw_calls.generic_gsw_caller(\
                                    'gsw_SA_from_SP', [vosaline_JP_level_PSU_north[:,i,...],pressure[i],lons[i],lats[i]]);

vosaline_JP_level_RS_north[:] = gsw_calls.generic_gsw_caller('gsw_SR_from_SP', [vosaline_JP_level_PSU_north[:]]);
votemper_JP_level_CT_north[:] = gsw_calls.generic_gsw_caller('gsw_CT_from_pt', [vosaline_JP_level_SA_north[:], votemper_JP_level_PT_north[:]]);        
    
print(vosaline_JP_level_RS_north.shape)


north_temp_function = interp1d(depth_JP_T, votemper_JP_level_CT_north, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
north_sal_function = interp1d(depth_JP_T, vosaline_JP_level_RS_north, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
north_U_function = interp1d(depth_JP_U, U_vel_JP_level_north,axis=1,\
                              bounds_error=False, fill_value='extrapolate');
north_V_function = interp1d(depth_JP_U, V_vel_JP_level_north, axis=1,\
                              bounds_error=False, fill_value='extrapolate');


votemper_NEMO_north = np.zeros((vosaline_JP_level_SA_north.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_north.shape[2], vosaline_JP_level_SA_north.shape[3]));
vosaline_NEMO_north = np.zeros((vosaline_JP_level_SA_north.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_north.shape[2], vosaline_JP_level_SA_north.shape[3]));
U_NEMO_north = np.zeros((vosaline_JP_level_SA_north.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_north.shape[2], vosaline_JP_level_SA_north.shape[3]));
V_NEMO_north = np.zeros((vosaline_JP_level_SA_north.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_north.shape[2], vosaline_JP_level_SA_north.shape[3]));


for indx in np.arange(NEMO_depth_T.shape[0]):
    votemper_NEMO_north[:,indx,...] = north_temp_function(NEMO_depth_T[indx]);
    vosaline_NEMO_north[:,indx,...] = north_sal_function(NEMO_depth_T[indx]);
    U_NEMO_north[:,indx,...]        = north_U_function(NEMO_depth_U[indx]);
    V_NEMO_north[:,indx,...]        = north_V_function(NEMO_depth_U[indx]);

    
print(votemper_NEMO_north.shape, vosaline_NEMO_north.shape, U_NEMO_north.shape, V_NEMO_north.shape)


#Now let us write the 3d boundary condition for the north boundary

bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_north_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xb', U_NEMO_north.shape[3]);
bdy_file.createDimension('yb', U_NEMO_north.shape[2]);
bdy_file.createDimension('deptht', U_NEMO_north.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along north boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of north boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = U_NEMO_north[:];
vomecrty[:] = V_NEMO_north[:];
votemper[:] = votemper_NEMO_north[:];
vosaline[:] = vosaline_NEMO_north[:];
deptht[:] = NEMO_depth_T[:];

bdy_file.close()

(43, 50, 4, 70) (43, 50, 4, 70) (43, 50, 4, 70) (43, 50, 4, 70)
(43, 50, 4, 70)
(43, 27, 4, 70) (43, 27, 4, 70) (43, 27, 4, 70) (43, 27, 4, 70)


### Now for the south boundary

In [36]:
south_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_south_m04.nc');

vosaline_JP_level_PSU_south = south_file.variables['vosaline'][:];
votemper_JP_level_PT_south = south_file.variables['votemper'][:];
U_vel_JP_level_south = south_file.variables['vozocrtx'][:];
V_vel_JP_level_south = south_file.variables['vomecrty'][:];


print(vosaline_JP_level_PSU_south.shape, votemper_JP_level_PT_south.shape, U_vel_JP_level_south.shape, V_vel_JP_level_south.shape)

vosaline_JP_level_SA_south = np.zeros(vosaline_JP_level_PSU_south.shape);
vosaline_JP_level_RS_south = np.zeros(vosaline_JP_level_PSU_south.shape);
votemper_JP_level_CT_south = np.zeros(votemper_JP_level_PT_south.shape);



pressure = gsw_calls.generic_gsw_caller('gsw_p_from_z.m', [z, np.mean(lat)]);

for i,j in enumerate(vosaline_JP_level_SA_south[0,:,...]):
    vosaline_JP_level_SA_south[:,i,...] = gsw_calls.generic_gsw_caller(\
                                    'gsw_SA_from_SP', [vosaline_JP_level_PSU_south[:,i,...],pressure[i],lons[i],lats[i]]);
vosaline_JP_level_RS_south[:] = gsw_calls.generic_gsw_caller('gsw_SR_from_SP', [vosaline_JP_level_PSU_south[:]]);
votemper_JP_level_CT_south[:] = gsw_calls.generic_gsw_caller('gsw_CT_from_pt', [vosaline_JP_level_SA_south[:], votemper_JP_level_PT_south[:]]);        
    
print(vosaline_JP_level_SA_south.shape)


south_temp_function = interp1d(depth_JP_T, votemper_JP_level_CT_south, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
south_sal_function = interp1d(depth_JP_T, vosaline_JP_level_RS_south, axis=1,\
                              bounds_error=False, fill_value='extrapolate');
south_U_function = interp1d(depth_JP_U, U_vel_JP_level_south,axis=1,\
                              bounds_error=False, fill_value='extrapolate');
south_V_function = interp1d(depth_JP_U, V_vel_JP_level_south, axis=1,\
                              bounds_error=False, fill_value='extrapolate');


votemper_NEMO_south = np.zeros((vosaline_JP_level_SA_south.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_south.shape[2], vosaline_JP_level_SA_south.shape[3]));
vosaline_NEMO_south = np.zeros((vosaline_JP_level_SA_south.shape[0], NEMO_depth_T.shape[0], \
                               vosaline_JP_level_SA_south.shape[2], vosaline_JP_level_SA_south.shape[3]));
U_NEMO_south = np.zeros((vosaline_JP_level_SA_south.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_south.shape[2], vosaline_JP_level_SA_south.shape[3]));
V_NEMO_south = np.zeros((vosaline_JP_level_SA_south.shape[0], NEMO_depth_U.shape[0], \
                               vosaline_JP_level_SA_south.shape[2], vosaline_JP_level_SA_south.shape[3]));


for indx in np.arange(NEMO_depth_T.shape[0]):
    votemper_NEMO_south[:,indx,...] = south_temp_function(NEMO_depth_T[indx]);
    vosaline_NEMO_south[:,indx,...] = south_sal_function(NEMO_depth_T[indx]);
    U_NEMO_south[:,indx,...]        = south_U_function(NEMO_depth_U[indx]);
    V_NEMO_south[:,indx,...]        = south_V_function(NEMO_depth_U[indx]);

    
print(votemper_NEMO_south.shape, vosaline_NEMO_south.shape, U_NEMO_south.shape, V_NEMO_south.shape)


#Now let us write the 3d boundary condition for the south boundary

bdy_file = nc.Dataset('/ocean/ssahu/CANYONS/bdy_files/3d_NEMO_south_m04.nc', 'w', zlib=True);


bdy_file.createDimension('xb', U_NEMO_south.shape[3]);
bdy_file.createDimension('yb', U_NEMO_south.shape[2]);
bdy_file.createDimension('deptht', U_NEMO_south.shape[1]);
bdy_file.createDimension('time_counter', None);


xb = bdy_file.createVariable('xb', 'int32', ('xb',), zlib=True);
xb.units = 'indices';
xb.longname = 'x indices along south boundary';

yb = bdy_file.createVariable('yb', 'int32', ('yb',), zlib=True);
yb.units = 'indices';
yb.longname = 'a strip of y indices across all of south boundary';

deptht = bdy_file.createVariable('deptht', 'float32', ('deptht',), zlib=True);
deptht.units = 'm';
deptht.longname = 'Vertical T Levels';

time_counter = bdy_file.createVariable('time_counter', 'int32', ('time_counter',), zlib=True);
time_counter.units = 's';
time_counter.longname = 'time';

vozocrtx = bdy_file.createVariable('vozocrtx', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vomecrty = bdy_file.createVariable('vomecrty', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
votemper = bdy_file.createVariable('votemper', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);
vosaline = bdy_file.createVariable('vosaline', 'float32', ('time_counter', 'deptht', 'yb', 'xb'), zlib=True);

vozocrtx[:] = U_NEMO_south[:];
vomecrty[:] = V_NEMO_south[:];
votemper[:] = votemper_NEMO_south[:];
vosaline[:] = vosaline_NEMO_south[:];
deptht[:] = NEMO_depth_T[:];

bdy_file.close()

(43, 50, 4, 70) (43, 50, 4, 70) (43, 50, 4, 70) (43, 50, 4, 70)
(43, 50, 4, 70)
(43, 27, 4, 70) (43, 27, 4, 70) (43, 27, 4, 70) (43, 27, 4, 70)
